In [1]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

df = pd.read_csv("C:\datasets/abalone.csv", index_col = 0)
df

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
id,,,,,,,,,
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9


In [2]:
for label in "MFI":
    df[label] = df["Sex"] == label
del df["Sex"]

df

,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings,M,F,I
id,,,,,,,,,,,
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15,True,False,False
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7,True,False,False
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9,False,True,False
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10,True,False,False
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...
4172,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11,False,True,False
4173,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10,True,False,False
4174,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9,True,False,False


In [3]:
X = df.drop('Rings', axis=1)
X.head()

,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,M,F,I
id,,,,,,,,,,
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,True,False,False
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,True,False,False
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,False,True,False
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,True,False,False
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,False,False,True


In [4]:
y = df['Rings']
y.value_counts()

9     689
10    634
8     568
11    487
7     391
12    267
6     259
13    203
14    126
5     115
15    103
16     67
17     58
4      57
18     42
19     32
20     26
3      15
21     14
23      9
22      6
27      2
24      2
1       1
26      1
29      1
2       1
25      1
Name: Rings, dtype: int64

In [5]:
y = pd.get_dummies(y).values

In [6]:
X = X.values

In [7]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [8]:
def split_sequence(X, y, n_steps):
    X_seq, y_seq = [], []
    for i in range(len(X)):
        end_ix = i + n_steps
        if end_ix > len(X) - 1:
            break
        X_seq.append(X[i:end_ix])
        y_seq.append(y[end_ix])
    return np.array(X_seq), np.array(y_seq)

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

n_steps = 3
X_seq, y_seq = split_sequence(X_scaled, y, n_steps)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

In [11]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3339, 3, 10), (835, 3, 10), (3339, 28), (835, 28))

In [12]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RNN

model = Sequential()

model.add(layers.Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=128, kernel_size=1, activation='relu'))
model.add(layers.GlobalMaxPooling1D())

model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(1))

model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [15]:
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
105/105 [==============================] - 0s 1ms/step - loss: 0.0345 - accuracy: 0.9643
Epoch 2/20
105/105 [==============================] - 0s 1ms/step - loss: 0.0345 - accuracy: 0.9643
Epoch 3/20
105/105 [==============================] - 0s 1ms/step - loss: 0.0344 - accuracy: 0.9643
Epoch 4/20
105/105 [==============================] - 0s 1ms/step - loss: 0.0345 - accuracy: 0.9643
Epoch 5/20
105/105 [==============================] - 0s 1ms/step - loss: 0.0345 - accuracy: 0.9643
Epoch 6/20
105/105 [==============================] - 0s 1ms/step - loss: 0.0345 - accuracy: 0.9643
Epoch 7/20
105/105 [==============================] - 0s 1ms/step - loss: 0.0345 - accuracy: 0.9643
Epoch 8/20
105/105 [==============================] - 0s 1ms/step - loss: 0.0346 - accuracy: 0.9643
Epoch 9/20
105/105 [==============================] - 0s 1ms/step - loss: 0.0345 - accuracy: 0.9643
Epoch 10/20
105/105 [==============================] - 0s 1ms/step - loss: 0.0345 - accuracy: 0.9643

In [14]:
y_pred = model.predict(X_test)
mse = np.mean((y_test - y_pred)**2)
print(f'Mean Squared Error: {mse}')

27/27 [==============================] - 0s 912us/step
Mean Squared Error: 0.03446153178811073
